In [1]:
#!/usr/bin/env python
# coding: utf-8

# Workflow
# 
# 1. read all csv.gz data by timestamp in file name, e.g. SSL_SDE-RG01a_202210090845_791308.csv -> 2021-10-09 08:45. currently read daily
# 2. parse data and then find date col and output the data in parquet format, partition by date col, in pipeline_output_dir
# 3. move processed data files to 'derived_dir'
# 4. Use sslweb_delete.py to delete files in derived dir daily

from smtutil import hiveutil, oracle_to_hive
from datetime import datetime, timedelta
import re, glob, shutil, os
from shutil import copyfile
from pathlib import Path
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import lag, lead, first, last, udf, lit, when, expr
import pyspark.sql.functions as F
import yaml
import argparse

# schema
schema_raw = StructType([
    StructField("MSISDN", StringType(), True),
    StructField("IP1", StringType(), True),
    StructField("TIME_START", StringType(), True),
    StructField("IP2", StringType(), True),
    StructField("BYTE_COUNT_UP", IntegerType(), True),
    StructField("BYTE_COUNT_DOWN", IntegerType(), True),
    StructField("SSL_CERT_DOMAIN", StringType(), False),
    StructField("TIME_END", StringType(), True)])

# get binary time string
def hourly_string(l):
    output = ''
    for i in range(0, 24):
        if int(i) in l:
            output += '1'
        else:
            output += '0'
    return output
# define pyspark udf
udf_hourly_string = F.udf(lambda x: hourly_string(x), StringType())
# binary string to integer
udf_b2int = F.udf(hiveutil.binary2int)

def move_file(input_dir, output_dir, filename_format):

    for f in Path(input_dir).glob(filename_format): # grabs all files
        f.rename(Path(output_dir).joinpath(f.name)) # moves to target folder.

    print(f'[Info] Moved files from {input_dir} to {output_dir} with file format {filename_format}')
    return

In [ ]:
# spark.stop()
# sc.stop()

In [8]:
cfg = '/app/sna_bdmrdev03/ws-simon/git_repo/weblog/ssl_to_parquet/config.yaml'
ts = 20230111

# load yaml
fhandle = open(cfg,'r')
cfg = yaml.safe_load(fhandle)

# add time stamp if needed
if ts != '':
    cfg['time_stamp'] = ts
    print(f'[Info] time stamp: {cfg["time_stamp"]}')

    print('[Info] Configs:\n')
    for k, v in cfg.items():
        print(f'{k} : {v}')
    print('\n')

[Info] time stamp: 20230111
[Info] Configs:

input_dir : /app/sna_bdmrdev01/t_ssl/raw/
working_dir : /app/sna_bdmrdev01/t_ssl/working/
pipeline_output_dir : /app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/
derived_dir : /app/sna_bdmrdev01/weblog_ssl_housekeep/
filename_format : *_{}*.csv.gz
byte_to_kb_ratio : 1024
time_stamp : 20230111




{'input_dir': '/app/sna_bdmrdev01/t_ssl/raw/',
 'working_dir': '/app/sna_bdmrdev01/t_ssl/working/',
 'pipeline_output_dir': '/app/sna_bdmrdev01/ws-ic/hive/prd_biz_summ_vw.db/vw_idp_ssl_sed_call_summ/',
 'derived_dir': '/app/sna_bdmrdev01/weblog_ssl_housekeep/',
 'filename_format': '*_{}*.csv.gz',
 'byte_to_kb_ratio': 1024,
 'time_stamp': 20230111}

In [10]:
# get spark session 
sc, spark = hiveutil.get_sc_spark()

# get time stamp
if cfg['time_stamp'] == '':
    cfg['time_stamp'] = datetime.strftime(datetime.now() - timedelta(days=1), '%Y%m%d')

# get file name format
filename_format = cfg['filename_format'].format(cfg['time_stamp'])
# print(f'[Info] Data Timestamp: {time_stamp}')
print(f'[Info] Filename Format: {filename_format}')

[Info] Filename Format: *_20230111*.csv.gz


In [25]:
time_stamp=cfg['time_stamp']
input_dir=cfg['input_dir']
working_dir=cfg['working_dir']
pipeline_output_dir=cfg['pipeline_output_dir']
derived_dir=cfg['derived_dir']
byte_to_kb_ratio=1024

In [12]:
### 1. -------------------------------------- Move file to working directory --------------------------------------
move_file(input_dir, working_dir, filename_format)

[Info] Moved files from /app/sna_bdmrdev01/t_ssl/raw/ to /app/sna_bdmrdev01/t_ssl/working/ with file format *_20230111*.csv.gz


In [13]:
### 2. -------------------------------------- Read data file --------------------------------------
print(f'[Info] Reading Data Files.....')
print(datetime.now())

# print(f'[Info] Read csv files of timestamp {time_stamp}')
# read csv
sdf = spark.read.csv(f"{working_dir}", sep='|', header=False, schema=schema_raw)
print(f'[Info] df size: {sdf.count()}, {len(sdf.columns)}')

# drop na domain rows
sdf = sdf.na.drop(subset=['SSL_CERT_DOMAIN'])
# print(f'[Info] df size after dropped na in Domain Column: {sdf.count()}, {len(sdf.columns)}')

# print(f'[Info] End of Reading Data Files')
# print(datetime.now())

[Info] Reading Data Files.....
2023-01-12 14:20:57.394619
[Info] df size: 2429077699, 8


In [14]:
### 3. -------------------------------------- Transform Data and Create Columns --------------------------------------

# create necessary columns for grouping
sdf = sdf.withColumn('ACCESS_DATE', F.to_date(F.col('TIME_START'), 'yyyyMMddHHmmss')) \
                     .withColumn('SSL_CERT_DOMAIN', F.upper(F.col('SSL_CERT_DOMAIN'))) \
                     .withColumn('START_TIMESTAMP', F.to_timestamp(F.col('TIME_START'),'yyyyMMddHHmmss')) \
                     .withColumn('END_TIMESTAMP', F.to_timestamp(F.col('TIME_END'),'yyyyMMddHHmmss')) \
                     .withColumn('SECONDS', F.col('END_TIMESTAMP').cast('long') - F.col('START_TIMESTAMP').cast('long')) \
                     .withColumn('MINUTES', F.col('SECONDS')/60) \
                     .withColumn('START_HOUR', F.hour(F.col('START_TIMESTAMP')))

In [20]:
print(sdf.count())
sdf.limit(1).toPandas()

MSISDN ACCESS_DATE       SSL_CERT_DOMAIN  TOTAL_HIT START_HOUR_SET  \
0  85219100007657  2023-01-11  BEACONS.GCP.GVT2.COM          1            [6]   

   BYTE_COUNT_UP  BYTE_COUNT_DOWN  SECONDS  MINUTES  
0           2188             7085      246      4.1

In [23]:
# group by SUBR_NUM, ACCESS_DATE, SSL_CERT_DOMAIN
sdf2 = sdf.groupBy('MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN').agg(
                F.count("*").alias('TOTAL_HIT'), \
                F.collect_set(F.col('START_HOUR')).alias('START_HOUR_SET'), \
                F.sum('BYTE_COUNT_UP').alias('BYTE_COUNT_UP'), \
                F.sum('BYTE_COUNT_DOWN').alias('BYTE_COUNT_DOWN'), \
                F.sum('SECONDS').alias('SECONDS'), \
                F.sum('MINUTES').alias('MINUTES')
            )

In [26]:
print(sdf2.count())
sdf2.limit(1).toPandas()

310143758


MSISDN ACCESS_DATE       SSL_CERT_DOMAIN  TOTAL_HIT START_HOUR_SET  \
0  85219100007657  2023-01-11  BEACONS.GCP.GVT2.COM          1            [6]   

   BYTE_COUNT_UP  BYTE_COUNT_DOWN  SECONDS  MINUTES  
0           2188             7085      246      4.1

In [27]:
# add last few columns
sdf3 = sdf2.withColumn('SUBR_NUM', when(F.col('MSISDN').startswith('852'), expr("substring(MSISDN, 4, length(MSISDN))")).otherwise(F.col('MSISDN'))) \
                     .withColumn('BYTE_COUNT', sum([F.col('BYTE_COUNT_UP'), F.col('BYTE_COUNT_DOWN')])) \
                     .withColumn('KB_COUNT', F.col('BYTE_COUNT')/byte_to_kb_ratio) \
                     .withColumn('KB_COUNT_UP', F.col('BYTE_COUNT_UP')/byte_to_kb_ratio) \
                     .withColumn('KB_COUNT_DOWN', F.col('BYTE_COUNT_DOWN')/byte_to_kb_ratio) \
                     .withColumn('HOUR_BINARY', udf_hourly_string(F.col('START_HOUR_SET'))) \
                     .withColumn('date_id', F.date_format(F.col('ACCESS_DATE'), "yyyyMMdd")).cache()

In [28]:
sdf3.limit(1).toPandas()

MSISDN ACCESS_DATE       SSL_CERT_DOMAIN  TOTAL_HIT START_HOUR_SET  \
0  85219100007657  2023-01-11  BEACONS.GCP.GVT2.COM          1            [6]   

   BYTE_COUNT_UP  BYTE_COUNT_DOWN  SECONDS  MINUTES     SUBR_NUM  BYTE_COUNT  \
0           2188             7085      246      4.1  19100007657        9273   

   KB_COUNT  KB_COUNT_UP  KB_COUNT_DOWN               HOUR_BINARY   date_id  
0  9.055664     2.136719       6.918945  000000100000000000000000  20230111

In [29]:
# reorder
#     sdf = sdf.select('SUBR_NUM', 'MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'KB_COUNT', 'HOUR_BINARY', 'TOTAL_HIT', 'BYTE_COUNT_UP', 'KB_COUNT_UP', 'BYTE_COUNT_DOWN', 'KB_COUNT_DOWN', 'SECONDS', 'MINUTES', 'date_id')
sdf4 = sdf3.select(F.col('SUBR_NUM').cast("bigint").alias("subr_num"), \
                 F.col('SSL_CERT_DOMAIN').alias("ssl_cert_domain"), \
                 F.col('BYTE_COUNT').cast("bigint").alias("byte_count"), \
                 udf_b2int(F.col('HOUR_BINARY')).cast("int").alias("hour_binary"), \
                 F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                 'date_id')

In [30]:
print(sdf4.count())
sdf4.limit(1).toPandas()

subr_num       ssl_cert_domain  byte_count  hour_binary  total_hit  \
0  19100007657  BEACONS.GCP.GVT2.COM        9273       131072          1   

    date_id  
0  20230111

In [31]:
# filter only today's data
sdf5 = sdf4.filter(F.col('date_id') == time_stamp)

# lower case column names
#     for col in sdf.columns:
#         sdf = sdf.withColumnRenamed(col, col.lower())

In [33]:
print(sdf5.count())
sdf5.limit(1).toPandas()

249688320


subr_num       ssl_cert_domain  byte_count  hour_binary  total_hit  \
0  19100007657  BEACONS.GCP.GVT2.COM        9273       131072          1   

    date_id  
0  20230111

In [37]:
sdf5.write.mode('append').partitionBy('date_id').parquet(f'{pipeline_output_dir}')
# sdf5.coalesce(10).write.mode('append').partitionBy('date_id').parquet(f'{pipeline_output_dir}')

In [2]:
def processer(
                spark,
                time_stamp,
                filename_format,
                input_dir,
                working_dir,
                pipeline_output_dir,
                derived_dir,
                byte_to_kb_ratio=1024,
            ):

    ### 1. -------------------------------------- Move file to working directory --------------------------------------
    move_file(input_dir, working_dir, filename_format)

    ### 2. -------------------------------------- Read data file --------------------------------------
    print(f'[Info] Reading Data Files.....')
    print(datetime.now())

    # print(f'[Info] Read csv files of timestamp {time_stamp}')
    # read csv
    sdf = spark.read.csv(f"{working_dir}", sep='|', header=False, schema=schema_raw)
    print(f'[Info] df size: {sdf.count()}, {len(sdf.columns)}')

    # drop na domain rows
    sdf = sdf.na.drop(subset=['SSL_CERT_DOMAIN'])
    # print(f'[Info] df size after dropped na in Domain Column: {sdf.count()}, {len(sdf.columns)}')

    # print(f'[Info] End of Reading Data Files')
    # print(datetime.now())

    ### 3. -------------------------------------- Transform Data and Create Columns --------------------------------------

    # create necessary columns for grouping
    sdf = sdf.withColumn('ACCESS_DATE', F.to_date(F.col('TIME_START'), 'yyyyMMddHHmmss')) \
                         .withColumn('SSL_CERT_DOMAIN', F.upper(F.col('SSL_CERT_DOMAIN'))) \
                         .withColumn('START_TIMESTAMP', F.to_timestamp(F.col('TIME_START'),'yyyyMMddHHmmss')) \
                         .withColumn('END_TIMESTAMP', F.to_timestamp(F.col('TIME_END'),'yyyyMMddHHmmss')) \
                         .withColumn('SECONDS', F.col('END_TIMESTAMP').cast('long') - F.col('START_TIMESTAMP').cast('long')) \
                         .withColumn('MINUTES', F.col('SECONDS')/60) \
                         .withColumn('START_HOUR', F.hour(F.col('START_TIMESTAMP')))

    # group by SUBR_NUM, ACCESS_DATE, SSL_CERT_DOMAIN
    sdf = sdf.groupBy('MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN').agg(
                    F.count("*").alias('TOTAL_HIT'), \
                    F.collect_set(F.col('START_HOUR')).alias('START_HOUR_SET'), \
                    F.sum('BYTE_COUNT_UP').alias('BYTE_COUNT_UP'), \
                    F.sum('BYTE_COUNT_DOWN').alias('BYTE_COUNT_DOWN'), \
                    F.sum('SECONDS').alias('SECONDS'), \
                    F.sum('MINUTES').alias('MINUTES')
                )

    # add last few columns
    sdf = sdf.withColumn('SUBR_NUM', when(F.col('MSISDN').startswith('852'), expr("substring(MSISDN, 4, length(MSISDN))")).otherwise(F.col('MSISDN'))) \
                         .withColumn('BYTE_COUNT', sum([F.col('BYTE_COUNT_UP'), F.col('BYTE_COUNT_DOWN')])) \
                         .withColumn('KB_COUNT', F.col('BYTE_COUNT')/byte_to_kb_ratio) \
                         .withColumn('KB_COUNT_UP', F.col('BYTE_COUNT_UP')/byte_to_kb_ratio) \
                         .withColumn('KB_COUNT_DOWN', F.col('BYTE_COUNT_DOWN')/byte_to_kb_ratio) \
                         .withColumn('HOUR_BINARY', udf_hourly_string(F.col('START_HOUR_SET'))) \
                         .withColumn('date_id', F.date_format(F.col('ACCESS_DATE'), "yyyyMMdd"))

    # reorder
#     sdf = sdf.select('SUBR_NUM', 'MSISDN', 'ACCESS_DATE', 'SSL_CERT_DOMAIN', 'BYTE_COUNT', 'KB_COUNT', 'HOUR_BINARY', 'TOTAL_HIT', 'BYTE_COUNT_UP', 'KB_COUNT_UP', 'BYTE_COUNT_DOWN', 'KB_COUNT_DOWN', 'SECONDS', 'MINUTES', 'date_id')
    sdf = sdf.select(F.col('SUBR_NUM').cast("bigint").alias("subr_num"), \
                     F.col('SSL_CERT_DOMAIN').alias("ssl_cert_domain"), \
                     F.col('BYTE_COUNT').cast("bigint").alias("byte_count"), \
                     udf_b2int(F.col('HOUR_BINARY')).cast("int").alias("hour_binary"), \
                     F.col("TOTAL_HIT").cast("bigint").alias("total_hit"), \
                     'date_id')
    
    # filter only today's data
    sdf = sdf.filter(F.col('date_id') == time_stamp)

    # lower case column names
#     for col in sdf.columns:
#         sdf = sdf.withColumnRenamed(col, col.lower())

    ### 4. -------------------------------------- Save to parquet --------------------------------------

    # print(f'[Info] Start Transforming Data......')
    # print(datetime.now())

    # save to parquet
    sdf.coalesce(10).write.mode('append').partitionBy('date_id').parquet(f'{pipeline_output_dir}')

    # print(f'[Info] Saved Data to {pipeline_output_dir}')
    # print(datetime.now())

    ### 5. -------------------------------------- Move Processed data to derived --------------------------------------

    # move processed files to derived folder
    move_file(working_dir, derived_dir, filename_format)

    # print(f"[Info] Moved Processed Data to {derived_dir}")
    print(f"[Info] Finished Processing Data with timestamp {time_stamp}")
    print(datetime.now())

    return

In [4]:
def main(
            filename_format,
            input_dir,
            working_dir,
            pipeline_output_dir,
            derived_dir,
            time_stamp='',
            byte_to_kb_ratio=1024,
        ):
    """
    time_stamp: timestamp to select data files to process
    filename_format: filename format, e.g. '*_{}*.csv.gz', used format to replace {} with timestamp
    input_dir: directory that contains the raw data from DW
    working_dir: directory that contains the processing data
    pipeline_output_dir: directory that contains the output parquet files
    derived_dir: directory that contains processed raw data from DW
    byte_to_kb_ratio: ratio for converting byte to kb
    """

    # get spark session 
    sc, spark = hiveutil.get_sc_spark()

    # get time stamp
    if time_stamp == '':
        time_stamp = datetime.strftime(datetime.now() - timedelta(days=1), '%Y%m%d')

    # get file name format
    filename_format = filename_format.format(time_stamp)
    # print(f'[Info] Data Timestamp: {time_stamp}')
    print(f'[Info] Filename Format: {filename_format}')

    # process data
    processer(
        spark=spark, 
        time_stamp=time_stamp,
        filename_format=filename_format,
        input_dir=input_dir,
        working_dir=working_dir,
        pipeline_output_dir=pipeline_output_dir,
        derived_dir=derived_dir
    )

    spark.stop()
    sc.stop()

    return

# if __name__ == '__main__':

#     # parser
#     parser = argparse.ArgumentParser(description='Self Defined Timestamp')
#     parser.add_argument('--ts', type=str, default='',
#                         help='timestamp of the date of data files')
#     parser.add_argument('--cfg', type=str, default='/app/sna_bdmrdev03/ws-simon/git_repo/weblog/ssl_to_parquet/testing/config_test.yaml',
#                         help='.yaml config file path')
#     args = parser.parse_args()

#     # load yaml
#     fhandle = open(args.cfg,'r')
#     cfg = yaml.safe_load(fhandle)

#     # add time stamp if needed
#     if args.ts != '':
#         cfg['time_stamp'] = args.ts
#         print(f'[Info] time stamp: {cfg["time_stamp"]}')

# #     print('[Info] Configs:\n')
# #     for k, v in cfg.items():
# #         print(f'{k} : {v}')
# #     print('\n')

#     main(**cfg)